Se plante el siguiente ejercicio: Limpieza de Datos

Examina cuidadosamente el archivo ‘Absenteeism-data.csv’. Luego, utiliza la siguiente guía para preparar los datos para un análisis posterior:

Elimina la columna ‘ID’.
Divide las razones de ausencia en múltiples variables dummym(dummy variables) y luego agrúpalas de la siguiente manera:
Grupo 1: Columnas 1 a 14
Grupo 2: Columnas 15, 16 y 17
Grupo 3: Columnas 18, 19, 20 y 21
Grupo 4: Columnas 22 a 28
Después de hacer eso, no olvides eliminar también la columna ‘Reason for Absence’.
Extrae el valor del mes y del día de la semana a partir de la columna ‘Date’. Luego, elimina también la columna ‘Date’.
Convierte los datos de la columna ‘Education’ en datos binarios, mapeando el valor 0 a 1, y el valor 1 y el resto de valores encontrados en esta columna a 0.
No olvides crear puntos de control a medida que avances. Si trabajaste correctamente, la versión final de tu DataFrame debería contener los mismos datos que los almacenados en el archivo ‘df-cleaned.csv’.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

INPUT_FILE  = Path("Absenteeism-data.csv")
OUTPUT_FILE = Path("df-cleaned-final.csv")


datos = pd.read_csv(INPUT_FILE)


col_id = [c for c in datos.columns if c.strip().lower() == "id"]
if col_id:
    datos = datos.drop(columns=col_id)

col_razon = next((c for c in datos.columns if "reason" in c.lower()), None)
if col_razon is None:
    raise ValueError("No se encontró la columna 'Reason for Absence'.")

categorias = list(range(1, 29))
razon_codificada = pd.Categorical(datos[col_razon], categories=categorias, ordered=False)
razon_dummies = pd.get_dummies(razon_codificada, prefix="R", prefix_sep="_")

def rango_cols(start, end):
    return [f"R_{i}" for i in range(start, end + 1)]

grupo_A = razon_dummies[rango_cols(1, 14)].max(axis=1).astype(int)
grupo_B = razon_dummies[rango_cols(15, 17)].max(axis=1).astype(int)
grupo_C = razon_dummies[rango_cols(18, 21)].max(axis=1).astype(int)
grupo_D = razon_dummies[rango_cols(22, 28)].max(axis=1).astype(int)

datos = datos.drop(columns=[col_razon]).assign(
    Grupo1=grupo_A,
    Grupo2=grupo_B,
    Grupo3=grupo_C,
    Grupo4=grupo_D
)


col_fecha = next((c for c in datos.columns if "date" in c.lower()), None)
if col_fecha is None:
    raise ValueError("No se encontró la columna 'Date'.")

fechas = pd.to_datetime(datos[col_fecha], errors="coerce", dayfirst=True)
datos["Mes"] = fechas.dt.month.astype("Int64")
datos["Dia_Semana"] = fechas.dt.weekday.astype("Int64")
datos = datos.drop(columns=[col_fecha])


col_educ = next((c for c in datos.columns if "education" in c.lower()), None)
if col_educ is None:
    raise ValueError("No se encontró la columna 'Education'.")

datos[col_educ] = datos[col_educ].map(lambda x: 1 if x == 0 else 0).astype("Int64")

# RESULTADO
datos.to_csv(OUTPUT_FILE, index=False)

print("✔ Limpieza completada.")
print(f"- Archivo generado: {OUTPUT_FILE}")

datos.head(10)


✔ Limpieza completada.
- Archivo generado: df-cleaned-final.csv


,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Grupo1,Grupo2,Grupo3,Grupo4,Mes,Dia_Semana
0,289,36,33,239.554,30,0,2,1,4,0,0,0,1,7,1
1,118,13,50,239.554,31,0,1,0,0,0,0,0,0,7,1
2,179,51,38,239.554,31,0,0,0,2,0,0,0,1,7,2
3,279,5,39,239.554,24,0,2,0,4,1,0,0,0,7,3
4,289,36,33,239.554,30,0,2,1,2,0,0,0,1,7,3
5,179,51,38,239.554,31,0,0,0,2,0,0,0,1,7,4
6,361,52,28,239.554,27,0,1,4,8,0,0,0,1,7,4
7,260,50,36,239.554,23,0,4,0,4,0,0,0,1,7,4
8,155,12,34,239.554,25,0,2,0,40,0,0,1,0,7,0
9,235,11,37,239.554,29,0,1,1,8,0,0,0,1,7,0
